In [1]:
import pandas as pd

In [2]:
# Step 1: Load the CSV file
df = pd.read_csv('../../data/cleaned_weather.csv', encoding='utf-8')

In [3]:
df.head()

,location_id,weather_code (wmo code),temperature_2m_max (°C),temperature_2m_min (°C),temperature_2m_mean (°C),apparent_temperature_max (°C),apparent_temperature_min (°C),apparent_temperature_mean (°C),daylight_duration (s),sunshine_duration (s),...,et0_fao_evapotranspiration (mm),latitude,longitude,elevation,year,month,day,sunrise_hour,sunset_hour,daylight_hours
0,0,1,30.1,22.6,26.0,34.5,25.0,29.0,42220.20,38905.73,...,4.61,6.924429,79.90725,4,2010,1,1,6,18,12
1,0,51,30.1,23.7,26.3,33.9,26.1,29.7,42225.71,37451.01,...,3.91,6.924429,79.90725,4,2010,2,1,6,18,12
2,0,51,29.6,23.1,26.0,34.5,26.2,29.9,42231.68,33176.43,...,3.66,6.924429,79.90725,4,2010,3,1,6,18,12
3,0,2,28.9,23.1,25.7,31.7,26.1,28.4,42238.11,38289.20,...,3.75,6.924429,79.90725,4,2010,4,1,6,18,12
4,0,1,28.1,21.3,24.6,30.0,22.9,26.2,42244.99,39113.82,...,5.00,6.924429,79.90725,4,2010,5,1,6,18,12


In [4]:
# Step 2: Create a proper 'date' column using year, month, day
df['date'] = pd.to_datetime(df[['year', 'month', 'day']], errors='coerce')

In [5]:
# Drop rows where 'date' is NaT due to invalid date creation
df = df.dropna(subset=['date'])

In [6]:
# Step 3: Columns you want to keep (rename for convenience)
columns_to_keep = {
    'date': 'date',
    'location_id': 'location_id',
    'temperature_2m_mean (°C)': 'temperature',
    'rain_sum (mm)': 'rainfall',
    'wind_speed_10m_max (km/h)': 'wind_speed',
    'precipitation_sum (mm)': 'precipitation'
}


In [7]:
# Step 4: Validate columns
missing = [col for col in columns_to_keep if col not in df.columns]
if missing:
    raise ValueError(f"The following expected columns are missing in the CSV: {missing}")

In [8]:
# Step 5: Create cleaned DataFrame
cleaned_df = df[list(columns_to_keep.keys())].rename(columns=columns_to_keep)

In [9]:
# Step 6: Drop rows with missing or invalid data
cleaned_df = cleaned_df.dropna()

In [10]:
# Step 7: Drop duplicates if any
cleaned_df = cleaned_df.drop_duplicates()

In [11]:
# Step 8: Sort by location and date
cleaned_df = cleaned_df.sort_values(by=['location_id', 'date']).reset_index(drop=True)

In [12]:
cleaned_df.head()   

,date,location_id,temperature,rainfall,wind_speed,precipitation
0,2010-01-01,0,26.0,0.0,12.2,0.0
1,2010-01-02,0,26.4,15.7,11.6,15.7
2,2010-01-03,0,28.0,0.0,13.0,0.0
3,2010-01-04,0,27.5,7.4,13.6,7.4
4,2010-01-05,0,27.7,9.6,10.3,9.6


In [13]:
# Step 9: Save cleaned CSV
cleaned_df.to_csv('cleaned_weather_data.csv', index=False)
print("Cleaned CSV saved as 'cleaned_weather_data.csv'")
print("Final columns:", cleaned_df.columns.tolist())

Cleaned CSV saved as 'cleaned_weather_data.csv'
Final columns: ['date', 'location_id', 'temperature', 'rainfall', 'wind_speed', 'precipitation']


In [14]:
# Check for outliers (basic IQR method)
numeric_cols = ['temperature', 'rainfall', 'wind_speed', 'precipitation']
for col in numeric_cols:
    q1 = cleaned_df[col].quantile(0.25)
    q3 = cleaned_df[col].quantile(0.75)
    iqr = q3 - q1
    outliers = cleaned_df[(cleaned_df[col] < (q1 - 1.5 * iqr)) | (cleaned_df[col] > (q3 + 1.5 * iqr))]
    print(f"{col}: {len(outliers)} outliers")


temperature: 9499 outliers
rainfall: 12986 outliers
wind_speed: 2181 outliers
precipitation: 12986 outliers
